In [ ]:
# в этом ноутбуке я считаю профиты для лонг трейдов со скользящим стоплоссом.
# собираюсь использовать это как раметку данных.
import pandas as pd
import numpy as np
import pandas_ta as ta
import forecast

trailing_stop_loss_percent = 0.1

In [ ]:
df = forecast.get_df('BTCUSD', '', '1T')

In [ ]:
def get_trailing_stop_loss_long_profits():
    opens = df['open'].to_numpy()
    highs = df['high'].to_numpy()
    lows = df['low'].to_numpy()
    closes = df['close'].to_numpy()

    n = len(opens)
    i = 0
    profits = []
    while i < n:
        #print(i)
        max = highs[i]
        max_i = i
        sl = max * (1 - 0.01 * trailing_stop_loss_percent)
        # starting from the current position look forward for trailing stop loss closing.
        j = i
        while j < n:
            # if there is a new max then save it, it's position and recalculate trailing stop loss.
            if max < highs[j]:
                max = highs[j]
                max_i = j
                sl = max * (1 - 0.01 * trailing_stop_loss_percent)
            # if price went lower than current stop loss then assume that the position was closed at the stop loss price.
            if lows[j] < sl or j == n - 1:
                profits.append(sl - opens[i])
                # all openings between the current opening and the max_i will be closed on the same stop loss position.
                while i < max_i:
                    i += 1
                    profits.append(sl - opens[i])
                break # break from while j loop.
            j += 1
        i += 1
    return profits

profits = get_trailing_stop_loss_long_profits()
# print(len(df))
# print(len(profits))

In [ ]:
df['profits'] = profits

import matplotlib.pyplot as plt
import mplfinance as mpf

customstyle = mpf.make_mpf_style(base_mpf_style='yahoo', facecolor='w')
dpi = 96
# магические числа 1.232 и 1.352 подобраны методом тыка, чтобы сохраненный png был ровно 1024x768.
# магические числа 1.285 и 1.352 подобраны методом тыка, чтобы сохраненный png был ровно 1536x768.
width_in_inches = (1024 * 1.232) / dpi
height_in_inches = (768 * 1.352) / dpi

df_slice = df[-1700:-1400]

apdict = [
    mpf.make_addplot(df_slice['open'] + df_slice['profits'], panel=0, color='black'),
    mpf.make_addplot(df_slice['open'] * (1 - trailing_stop_loss_percent * 0.01), panel=0, color='blue'),
    mpf.make_addplot(df_slice['profits'], panel=2, color='black'),
    ]

# отрисовка.
fig, ax = mpf.plot(
    df_slice,
    type='candle',
    volume=True, 
    style=customstyle,
    figsize=(width_in_inches, height_in_inches),
    returnfig=True, # это надо, чтобы mpf.plot вернул fix, ax.
    # разворот подписей по оси X, т.к. в документации openai написано, что повернутый текст он плохо читает (наверное 
    # умышленно порезана модель, чтобы нельзя было ее использовать для распознавания капчи)
    xrotation=0, 
    #return_width_config=wconfig,
    update_width_config={'volume_width': 0.75, 'candle_width': 0.75, 'candle_linewidth': 1},
    addplot=apdict
    )
